# Feature Engineering Part 1

On this Noteboook, the process of Feature Enginnering and Feature Selection is debugged using reduced data. After the definition of the process, all the code generated is applied to the complete data on Part 2.

## Libraries

In [1]:
import numpy as np
import pandas as pd
from cnr_methods import get_simplified_data, transform_data, rfe_score

# Feature Engineering Library for Time Series
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute
from patsy import dmatrix

# Feature Selection Libraries
from cnr_methods import LOFO_GPU_Importance
import xgboost as xgb

#Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

## Read Data

For this pipeline, only Training Set will be used.

In [2]:
full_data, y_train = get_simplified_data()
train_data = full_data[full_data['Set']=='Train']

In [3]:
train_data.head()

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.220571,-0.266871,0.997093,-1.415138,286.575,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460,95.905879,Train


To simplify the work, we will generate features for just one Wind Farm. When doing modelling, the features, as the models, will be generated for all Wind Farms separately.

In [4]:
WF = 'WF1'
data = train_data[train_data['WF']==WF]
y_train = y_train[y_train.index.isin(data['ID'])]

In [5]:
data

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440000,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260000,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.220571,-0.266871,0.997093,-1.415138,286.575000,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780000,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460000,95.905879,Train
...,...,...,...,...,...,...,...,...,...
2019-01-15 20:00:00,6235,WF1,-2.038990,-7.244520,-0.077702,-2.412487,280.165000,0.000000,Train
2019-01-15 21:00:00,6236,WF1,-1.453913,-5.145231,-0.291905,-2.148108,280.748363,0.000000,Train
2019-01-15 22:00:00,6237,WF1,-2.378300,-5.825480,-0.559868,-1.926407,279.299000,0.000000,Train


## Feature Creation

First, using the Zonal and Meridional Components of Wind, the Magnitude and Direction of Wind Vector for 100m and 10m height.

### Wind Speed Vector

In [6]:
feature_data = data[['ID','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]
feature_data['Wind Speed 100m'] = np.sqrt(feature_data['U_100m']**2 + feature_data['V_100m']**2)
feature_data['Wind Direction 100m'] = np.arctan(feature_data['V_100m']/feature_data['U_100m'])
feature_data['Wind Speed 10m'] = np.sqrt(feature_data['U_10m']**2 + feature_data['V_10m']**2)
feature_data['Wind Direction 10m'] = np.arctan(feature_data['V_10m']/feature_data['U_10m'])

Changing Reference for Negative Angles:

In [7]:
feature_data['Wind Direction 100m'] = feature_data['Wind Direction 100m'].apply(lambda x: 360 + x if x < 0 else x)
feature_data['Wind Direction 10m'] = feature_data['Wind Direction 10m'].apply(lambda x: 360 + x if x < 0 else x)

Using the original Features, we will create some variables over the Numerical Variables from the simplified data.

In [8]:
features = ['T', 'CLCT', 'U_100m','V_100m','U_10m','V_10m']

### Time-Relative Variables

Here,  Values for Last Week and Month for each Numerical Feature are generated.

In [9]:
for column in features:
    feature_data[column + '_lag_7_days'] = feature_data[column].shift(7)
    feature_data[column + '_lag_14_days'] = feature_data[column].shift(14)
    feature_data[column + '_lag_21_days'] = feature_data[column].shift(21)

Now, Month and Quarter Statistics(Mean,Median,Variance) are generated. It's important to remember here that, during a month or quarter, only passed Months or Quarters data is known. So, to avoid Leakage, the last Month and Quarter is used for each row.

In [10]:
feature_data['Month_Number'] = feature_data.index.month

In [11]:
# Month
mean_month = feature_data.groupby('Month_Number').mean()[features]
variance_month = feature_data.groupby('Month_Number').var()[features]

In [12]:
# Month
mean_month.columns = mean_month.columns + '_Last_Month_Mean'
variance_month.columns = variance_month.columns + 'Last_Month_Variance'

In [13]:
# Month
month_data = mean_month.merge(variance_month,on='Month_Number',how='left')

In [14]:
# Month
month_data = month_data.reset_index()
month_data['Month_Number'] = month_data['Month_Number'] + 1
month_data['Month_Number'] = month_data['Month_Number'].replace({13:1})

In [15]:
# Month
feature_data = feature_data.merge(month_data,on='Month_Number',how='left')

In [16]:
feature_data.index = data.index

For periodical Features, here represented by days (Of Month, Week and Year), hour and minutes, the features are applied to sinusoidal functions to replicate the cyclic nature of the variables.

In [17]:
day = feature_data.index.day
hour = feature_data.index.hour
minute = feature_data.index.minute
dayofweek = feature_data.index.dayofweek
dayofyear = feature_data.index.dayofyear

In [18]:
days_in_month = feature_data.index.days_in_month

In [19]:
feature_data["cos_day"], feature_data["sin_day"] = (
    np.cos(2 * np.pi * (day - 1) / days_in_month),
    np.sin(2 * np.pi * (day - 1) / days_in_month),
    )

feature_data["cos_hour"], feature_data["sin_hour"] = (
    np.cos(2 * np.pi * hour / 24),
    np.sin(2 * np.pi * hour / 24),
    )

feature_data["cos_minute"], feature_data["sin_minute"] = (
    np.cos(2 * np.pi * minute / 60),
    np.sin(2 * np.pi * minute / 60),
)

feature_data["cos_dayofyear"], feature_data["sin_dayofyear"] = (
    np.cos(2 * np.pi * (dayofyear - 1) / 365),
    np.sin(2 * np.pi * (dayofyear - 1) / 365),
)

feature_data["cos_dayofweek"], feature_data["sin_dayofweek"] = (
    np.cos(2 * np.pi * dayofweek / 7),
    np.sin(2 * np.pi * dayofweek / 7),
)

### Distance from Features

Distance of Position of Max and Min (Already on Tsfresh, check it later):

In [20]:
for column in features:
    feature_data[column + '_Distance_Max'] = feature_data.index - feature_data[column].idxmax()
    feature_data[column + '_Distance_Min'] = feature_data.index - feature_data[column].idxmin()
    feature_data[column + '_Distance_Max'] = feature_data[column + '_Distance_Max'].apply(lambda x : x.days)
    feature_data[column + '_Distance_Min'] = feature_data[column + '_Distance_Min'].apply(lambda x : x.days)

### Rolling Window Statistics

In [21]:
for column in features:
    feature_data[column + '_Rolling_7_Window_Mean'] = feature_data[column].rolling(window=7).mean()
    feature_data[column + '_Rolling_14_Window_Mean'] = feature_data[column].rolling(window=14).mean()
    feature_data[column + '_Rolling_7_Window_Variance'] = feature_data[column].rolling(window=7).var()
    feature_data[column + '_Rolling_14_Window_Variance'] = feature_data[column].rolling(window=14).var()

### Expanding Window Statistics

In [22]:
for column in features:
    #feature_data[column + '_Expanded_Window_Min'] = feature_data[column].expanding().min()
    feature_data[column + '_Expanded_Window_Max'] = feature_data[column].expanding().max()

### Natural Spline Features

In [23]:
for column in features:
    splines = dmatrix("cr(base_features,df=4,lower_bound=0,upper_bound=len(base_features)-1)", {"base_features": feature_data[column]}, return_type='dataframe')
    feature_data[column + '_Spline_0'] = splines.iloc[:,1]
    feature_data[column + '_Spline_1'] = splines.iloc[:,2]
    #feature_data[column + '_Spline_2'] = splines.iloc[:,3]

In [24]:
import matplotlib.pyplot as plt

In [25]:
feature = 'U_100m'
plt.figure(figsize=(10,8))
plt.plot(feature_data.index,feature_data[feature+'_Spline_0'],label='Spline 0')
plt.plot(feature_data.index,feature_data[feature+'_Spline_1'],label='Spline 1')
plt.plot(feature_data.index,feature_data[feature+'_Spline_2'],label='Spline 2')
plt.legend()

KeyError: 'U_100m_Spline_2'

In [26]:
feature_data

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set,Wind Speed 100m,...,CLCT_Spline_0,CLCT_Spline_1,U_100m_Spline_0,U_100m_Spline_1,V_100m_Spline_0,V_100m_Spline_1,U_10m_Spline_0,U_10m_Spline_1,V_10m_Spline_0,V_10m_Spline_1
Time,,,,,,,,,,,,,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440000,82.543144,Train,3.958410,...,0.105804,-0.358382,2.436560,-1.609720,2.941984,-2.236338,-0.011748,1.006061,1.220879,-0.249896
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260000,99.990844,Train,2.831607,...,0.309034,-1.046768,2.555395,-1.742879,1.862025,-0.992685,-0.121232,0.507772,1.315184,-0.356590
2018-05-01 03:00:00,3,WF1,-1.220571,-0.266871,0.997093,-1.415138,286.575000,98.367235,Train,1.249405,...,0.290196,-0.982957,1.779819,-0.873817,1.159083,-0.183195,0.133100,0.905004,2.079007,-1.220757
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780000,94.860604,Train,7.238384,...,0.249458,-0.844968,-0.120402,0.527478,4.706211,-4.267975,0.361583,0.691139,1.733075,-0.829379
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460000,95.905879,Train,6.647232,...,0.261608,-0.886126,-0.108358,0.467113,4.245543,-3.737481,0.718244,0.312389,1.224902,-0.254447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-15 20:00:00,6235,WF1,-2.038990,-7.244520,-0.077702,-2.412487,280.165000,0.000000,Train,7.525992,...,1.000000,0.000000,2.302705,-1.459730,5.318480,-4.973048,1.075950,-0.084615,2.839461,-2.081112
2019-01-15 21:00:00,6236,WF1,-1.453913,-5.145231,-0.291905,-2.148108,280.748363,0.000000,Train,5.346706,...,1.000000,0.000000,1.928901,-1.040869,4.067088,-3.531977,1.285323,-0.317875,2.637878,-1.853047
2019-01-15 22:00:00,6237,WF1,-2.378300,-5.825480,-0.559868,-1.926407,279.299000,0.000000,Train,6.292259,...,1.000000,0.000000,2.519489,-1.702645,4.472586,-3.998938,1.547243,-0.609676,2.468837,-1.661799


In [27]:
feature_data['T_Spline_2']

KeyError: 'T_Spline_2'

## Feature Selection

### LOFO Selection

Finally, all the Features Generated have to be filtered, so only the most relevant ones are passed to the model, as a way of avoiding Overfitting. For this process, a process called LOFO (Leave One Feature Out) is selected.

The process is simple: For each Feature, a arbitrary Model (Here a XGBoost) is cross validated on a dataset that contains all the features except one, which importance is being measured, and the precision without that feature is compared to a baseline, where all features are present. The features whose removal leads to worse results are considered most important features to the dataset.

More info about this process can be found on https://github.com/aerdem4/lofo-importance.

The implementation of this method on this however, was an adaptation of the original code to allow use of GPU resources as a faster way to obtain results, since this work involves a big number of columns.

In [28]:
final_features = feature_data.drop(features,axis=1)
final_features = final_features.rename({'key_0':'Date'},axis=1)

In [29]:
features_list = final_features.drop(['ID','WF','Set'],axis=1).columns

In [30]:
param = {'tree_method' : 'gpu_hist'}

In [31]:
importance_df = LOFO_GPU_Importance(final_features,y_train,features_list,param)

1/99 3.353021 s/it
2/99 3.394884 s/it
3/99 3.142598 s/it
4/99 3.143206 s/it
5/99 3.257639 s/it
6/99 3.447000 s/it
7/99 2.792045 s/it
8/99 3.460757 s/it
9/99 3.435719 s/it
10/99 3.701389 s/it
11/99 3.346872 s/it
12/99 3.452518 s/it
13/99 3.163006 s/it
14/99 3.968924 s/it
15/99 3.146929 s/it
16/99 3.461841 s/it
17/99 3.361761 s/it
18/99 3.165512 s/it
19/99 3.50622 s/it
20/99 3.758414 s/it
21/99 3.371025 s/it
22/99 4.432042 s/it
23/99 2.993312 s/it
24/99 3.34151 s/it
25/99 3.55543 s/it
26/99 3.14016 s/it
27/99 3.10023 s/it
28/99 3.34879 s/it
29/99 3.1622 s/it
30/99 3.34051 s/it
31/99 3.24403 s/it
32/99 3.19262 s/it
33/99 3.72313 s/it
34/99 3.23490 s/it
35/99 3.71034 s/it
36/99 3.58350 s/it
37/99 3.82508 s/it
38/99 3.695326 s/it
39/99 2.930569 s/it
40/99 3.46001 s/it
41/99 3.50375 s/it
42/99 3.254786 s/it
43/99 3.27363 s/it
44/99 3.64511 s/it
45/99 3.63529 s/it
46/99 3.11670 s/it
47/99 3.267825 s/it
48/99 3.22513 s/it
49/99 3.361135 s/it
50/99 3.91232 s/it
51/99 3.48001 s/it
52/99 2.997998

In [32]:
importance_df.to_csv(r'C:\Users\andre_\OneDrive\Documentos\Feature Selection\Importance_WF1.csv')

In [33]:
importance_df

,feature,score
3,Wind Direction 10m,-2.940089
94,V_100m_Spline_1,-1.816084
17,U_10m_lag_14_days,-1.379134
68,U_100m_Rolling_14_Window_Variance,-1.284125
1,Wind Direction 100m,-1.245187
...,...,...
9,CLCT_lag_21_days,0.481689
70,V_100m_Rolling_14_Window_Mean,0.526366
4,T_lag_7_days,0.634640
13,V_100m_lag_7_days,1.169991


### RFE Feature Selection

In [34]:
param = {'tree_method' : 'gpu_hist'}
num_boost_round = 1000
early_stopping_rounds = 100

In [35]:
final_features = final_features.drop(['ID','WF','Set'],axis=1)

In [36]:
selected_features = rfe_score(final_features,y_train,param,num_boost_round,early_stopping_rounds)

In [37]:
selected_features

0                      Wind Speed 100m
1                  Wind Direction 100m
2                       Wind Speed 10m
3                        T_lag_14_days
4                        T_lag_21_days
5                   Wind Direction 10m
6                         T_lag_7_days
7                   U_100m_lag_21_days
8      U_10m_Rolling_7_Window_Variance
9     V_100m_Rolling_7_Window_Variance
10                  V_100m_lag_14_days
11                    CLCT_lag_21_days
12                    U_10m_lag_7_days
13                   U_100m_lag_7_days
14    U_10m_Rolling_14_Window_Variance
Name: feature, dtype: object

## Full Generation of Features

Here, the features created above are generated for all the data.

### Functions

In [38]:
def get_manual_features(feature_data):

    index = feature_data.index
    features = ['T', 'CLCT', 'U_100m','V_100m','U_10m','V_10m']

    # Wind Speed Vector
    feature_data['Wind Speed 100m'] = np.sqrt(feature_data['U_100m']**2 + feature_data['V_100m']**2)
    feature_data['Wind Direction 100m'] = np.arctan(feature_data['V_100m']/feature_data['U_100m'])
    feature_data['Wind Speed 10m'] = np.sqrt(feature_data['U_10m']**2 + feature_data['V_10m']**2)
    feature_data['Wind Direction 10m'] = np.arctan(feature_data['V_10m']/feature_data['U_10m'])

    feature_data['Wind Direction 100m'] = feature_data['Wind Direction 100m'].apply(lambda x: 360 + x if x < 0 else x)
    feature_data['Wind Direction 10m'] = feature_data['Wind Direction 10m'].apply(lambda x: 360 + x if x < 0 else x)

    # Time Relative Variables 

    for column in features:
        feature_data[column + '_lag_7_days'] = feature_data[column].shift(7)
        feature_data[column + '_lag_14_days'] = feature_data[column].shift(14)
        feature_data[column + '_lag_21_days'] = feature_data[column].shift(21)

    feature_data['Month_Number'] = feature_data.index.month # Month Number

    mean_month = feature_data.groupby('Month_Number').mean()[features]
    variance_month = feature_data.groupby('Month_Number').var()[features]

    mean_month.columns = mean_month.columns + '_Last_Month_Mean'
    variance_month.columns = variance_month.columns + 'Last_Month_Variance'

    month_data = mean_month.merge(variance_month,on='Month_Number',how='left')
    month_data = month_data.reset_index()
    month_data['Month_Number'] = month_data['Month_Number'] + 1
    month_data['Month_Number'] = month_data['Month_Number'].replace({13:1})

    feature_data = feature_data.merge(month_data,on='Month_Number',how='left')
    feature_data.index = index

    # Periodical Features

    day = feature_data.index.day
    hour = feature_data.index.hour
    minute = feature_data.index.minute
    dayofweek = feature_data.index.dayofweek
    dayofyear = feature_data.index.dayofyear
    days_in_month = feature_data.index.days_in_month

    feature_data["cos_day"], feature_data["sin_day"] = (
    np.cos(2 * np.pi * (day - 1) / days_in_month),
    np.sin(2 * np.pi * (day - 1) / days_in_month),
    )

    feature_data["cos_hour"], feature_data["sin_hour"] = (
        np.cos(2 * np.pi * hour / 24),
        np.sin(2 * np.pi * hour / 24),
        )

    feature_data["cos_minute"], feature_data["sin_minute"] = (
        np.cos(2 * np.pi * minute / 60),
        np.sin(2 * np.pi * minute / 60),
    )

    feature_data["cos_dayofyear"], feature_data["sin_dayofyear"] = (
        np.cos(2 * np.pi * (dayofyear - 1) / 365),
        np.sin(2 * np.pi * (dayofyear - 1) / 365),
    )

    feature_data["cos_dayofweek"], feature_data["sin_dayofweek"] = (
        np.cos(2 * np.pi * dayofweek / 7),
        np.sin(2 * np.pi * dayofweek / 7),
    )

    # Distance from Max and Min

    for column in features:
        feature_data[column + '_Distance_Max'] = feature_data.index - feature_data[column].idxmax()
        feature_data[column + '_Distance_Min'] = feature_data.index - feature_data[column].idxmin()
        feature_data[column + '_Distance_Max'] = feature_data[column + '_Distance_Max'].apply(lambda x : x.days)
        feature_data[column + '_Distance_Min'] = feature_data[column + '_Distance_Min'].apply(lambda x : x.days)

    # Rolling Window Statistics

    for column in features:
        feature_data[column + '_Rolling_7_Window_Mean'] = feature_data[column].rolling(window=7).mean()
        feature_data[column + '_Rolling_14_Window_Mean'] = feature_data[column].rolling(window=14).mean()
        feature_data[column + '_Rolling_7_Window_Variance'] = feature_data[column].rolling(window=7).var()
        feature_data[column + '_Rolling_14_Window_Variance'] = feature_data[column].rolling(window=14).var()

    # Expanded Window Statistics

    for column in features:
        #feature_data[column + '_Expanded_Window_Min'] = feature_data[column].expanding().min()
        feature_data[column + '_Expanded_Window_Max'] = feature_data[column].expanding().max()

    # Spline Features

    for column in features:
        splines = dmatrix("cr(base_features,df=3,lower_bound=0,upper_bound=len(base_features)-1)", {"base_features": feature_data[column]}, return_type='dataframe')
        feature_data[column + '_Spline_0'] = splines.iloc[:,1]
        feature_data[column + '_Spline_1'] = splines.iloc[:,2]
        #feature_data[column + '_Spline_2'] = splines.iloc[:,3]

    # Dropping Base Features 
    #features.append(['Month_Number','Quarter Number'])
    feature_data = feature_data.drop(features,axis=1)

    return feature_data

In [39]:
def get_features_data(X): 
    feature_data = pd.DataFrame()
    for WF in X['WF'].unique():
        X_WF = X[X['WF']==WF]

        X_WF = get_manual_features(X_WF)

        X_WF['WF'] = WF
        feature_data = pd.concat([feature_data,X_WF],axis=0)

    feature_data = pd.concat([X,feature_data],axis=1) 

    return feature_data

### Process Aplication

In [40]:
X,y_train = get_simplified_data()

X_train = X[X['Set']=='Train']
X_test = X[X['Set']=='Test']

In [41]:
X_train = get_features_data(X_train)
X_test = get_features_data(X_test)

In [42]:
X_train['Set'] = 'Train'
X_test['Set'] = 'Test'

feature_data = pd.concat([X_train,X_test],axis=0)

feature_data = feature_data.loc[:,~feature_data.columns.duplicated()]


feature_data.to_csv(r'C:\Users\andre_\OneDrive\Documentos\Feature Selection\Selected_Features_Data.csv')